<a href="https://colab.research.google.com/github/sylvia31096/Histopathologic-Cancer-Detection/blob/master/Histopathologic_Test_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fetching dataset from Kaggle

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sylviachach","key":"4239815dc7975611718ace30d98bfb28"}'}

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!kaggle competitions download -c histopathologic-cancer-detection

  0% 0.00/1.33M [00:00<?, ?B/s]
100% 1.33M/1.33M [00:00<00:00, 44.2MB/s]
 98% 5.00M/5.10M [00:00<00:00, 39.2MB/s]
100% 5.10M/5.10M [00:00<00:00, 32.6MB/s]
100% 1.30G/1.30G [00:10<00:00, 111MB/s] 
100% 1.30G/1.30G [00:10<00:00, 133MB/s]
100% 4.97G/4.98G [00:50<00:00, 151MB/s]
100% 4.98G/4.98G [00:50<00:00, 105MB/s]


Import required libraries

In [0]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
from keras import backend as K
from keras.preprocessing import image

from keras.models import Sequential,load_model
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D
import os
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


Extract the zip files 

In [0]:
with ZipFile("/content/train.zip","r") as zip_ref:
    zip_ref.extractall("content/train")

Get the list of file names

In [0]:
with ZipFile("train.zip", "r") as f:
   listOfFiles= [name for name in f.namelist()]

Join the filenames and the corresponding targets.

In [0]:
#get the targets
targets = pd.read_csv('train_labels.csv.zip')
targets = targets.set_index('id')
filenames = pd.DataFrame(listOfFiles,[os.path.splitext(base)[0] for base in listOfFiles])
filetarg = pd.concat([filenames, targets], axis=1,join='inner')


Rename the columns

In [0]:
filetarg.columns= ['path','label']


In [0]:
filetarg = filetarg.applymap(str)
filetarg.dtypes

path     object
label    object
dtype: object

Pass the filetarg dataframe to the ImageDataGenerator for:


1.   Passing images to the model in batches
2.   Split into validation and training



In [0]:
train_datagen = image.ImageDataGenerator(validation_split=0.2)
train_generator = train_datagen.flow_from_dataframe(filetarg, directory='content/train',
              x_col='path',
              y_col='label',
              batch_size=450,
              target_size=(96, 96),
              subset="training" 
             )
validation_generator = train_datagen.flow_from_dataframe(filetarg, directory='content/train',
              x_col='path',
              y_col='label',
              batch_size=450,
              target_size=(96, 96),
              subset="validation"                                           
             )

Found 176020 images belonging to 2 classes.
Found 44005 images belonging to 2 classes.


Get the saved model from google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Retrieve test data

In [0]:
with ZipFile("test.zip", "r") as f:
   listOfFiles= [name for name in f.namelist()]
testdf = pd.DataFrame(listOfFiles)


In [0]:
with ZipFile("/content/test.zip","r") as zip_ref:
    zip_ref.extractall("content/test/test")

Pass the testdf dataframe to the ImageDataGenerator to pass images to the model in batches.



In [0]:

test_datagen = image.ImageDataGenerator(rescale=1./255,
                                         shear_range=0.2,
                                         zoom_range=0.2,
                                         horizontal_flip=True)

test_generator = test_datagen.flow_from_directory(
        'content/test',
        target_size=(96, 96),
        color_mode="rgb",
        shuffle = False,
        class_mode='binary',
        batch_size=500)
#test_generator.reset()


Found 57458 images belonging to 1 classes.


Predict the test data.

In [0]:
predictions = model1.predict_generator(test_generator, steps=115, verbose=0)


In [0]:
predicted_class_indices=np.argmax(predictions,axis=1)



array([1, 1, 1, ..., 0, 0, 0])

In [0]:
labels = (train_generator.class_indices)


{'0': 0, '1': 1}

In [0]:
labels = dict((v,k) for k,v in labels.items())


{0: '0', 1: '1'}

In [0]:
predictions = [labels[k] for k in predicted_class_indices]


['1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0'

In [0]:
prediction=predictions.reshape(2,57458)

Save in a csv file

In [0]:
filenames=test_generator.filenames
#filenames = listOfFiles
results=pd.DataFrame({"id":filenames,
                      "label":X_1})
#results['id'] = results['id'].str.rstrip('.tif')
results['id'] = results['id'].str[:-4]
results['id'] = results['id'].str[5:]

results.to_csv("results.csv",index=False)